# 赛事说明

**赛事官方：**

地址：https://data.xm.gov.cn/contest-series/period/#/5/

**赛题要求：**

参赛者需要基于主办方提供的综合信息数据，对信息数据进行分类，通过模型建立、语义分析等方法筛选出食品安全相关的信息，输出属于食品安全相关的信息编号及信息名称，以助力相关部门监管高效精准。 针对食品及食品安全名词定义如下： (1) 食品，指各种供人食用或者饮用的成品和原料以及按照传统既是食品又是中药材的物品，但是不包括以治疗为目的的物品。 (2) 食品安全，指食品无毒、无害，符合应当有的营养要求，对人体健康不造成任何急性、亚急性或者慢性危害。食品安全包括食品卫生、食品质量、食品营养等相关内容和食品（食物）种植、养殖、加工、包装、储藏、运输、销售、消费等各个环节。

**提交格式要求**
算法运行结果保存在文件名为result.txt，格式为 utf-8。
算法运行结果文件内容中，每个字段间以英文半角竖线 “|” 符 分隔。如下:
```
EVENT_ID|EVENT_TYPE|EVENT_NAME
Y000885|0|报警人报称衣服被施工的泥土搞脏了,现**工单位因赔偿问题有争执。
Y003481|1|请**官方展开卫生调查
```
输出结果字段注释

|字段名|字段释意|备注|
|---|---|---|
|EVENT_ID|信息唯一编号|{信息 ID_001，信息 ID_002，…}|
|EVENT_TYPE|是否属于食品安全相关信息|{0：否；1：是}|
|EVENT_NAME|信息名称|信息唯一编号所对应的信息标题|



**baseline思路**

本赛题是基于无监督数据的二元文本分类问题

最大难点在于所给数据集中没有任何标注

- 1 **规则匹配**（参考Betterme大佬0.94的baseline）(召回高，精确低)
- 2 使用**TextRank**筛选TopK个关键词，使用这些关键词进行规则匹配(召回低，精确高)
- 3 遵循赛题构建分类模型，baseline使用**异常检测模型**做一个分类器，区别与常规需要二分类数据才能构建异常检测模型，本Baseline选择**One-Class SVM分类器**，OCSVM在训练时只需要一种标签的文本（即正例，比赛中是食品安全有关的文本）进行训练，其思想是利用正例数据在向量空间中拟合出一个封闭的边界，边界内的是正例，边界外的是负例（异常样本）。(平衡召回和精确)
    其中存在两个问题：
    - **如何提取文本向量？** 在paddlenlp中调用预训练模型获得文本的句向量，本baseline使用sim-bert。
    - **没有真实标签如何判定分类器训练的好坏？** 分类器的决策边界取决于正例的数量以及超参数的设置，超参数惩罚严格得到边界太小，召回率会降低，超参数惩罚宽松边界过大，精确率会降低，目前暂未想到比较好的处理方式。由于TextRank匹配比规则匹配更严格，得到的正例是很少，默认超参数下分类器的边界太小，所以baseline中的方式是把规则匹配的结果当做伪真实标签，规则匹配能够召回大量的标签，通过调参的方式尽量让分类器的结果在F1值上贴近规则匹配的结果，以达到慢慢扩大决策边界平衡召回率和精确率。当然，结果上可能很难超过规则匹配的分数，并且有些耍小聪明的感觉，要是官方没有明确说7000条数据中食品安全数据只有1500条，那1500的界限也很难估计出来的，所以baseline中这种判定分类器好坏的方式仅在本场比赛适用。
- 4 使用训练好的OCSVM模型，推理所有数据，得到提交结果。

后续优化：
- 优化句向量，使用更SOTA的句向量表征模型（...）
- 优化异常检测模型，使用更SOTA的异常检测模型（..）
- 优化规则，增加更多的规则（.）
- 优化伪标签，使用弱监督方法、半监督方法利用少量可靠的人工标注/筛选标签构建文本分类器（..）
- 使用文本匹配代替文本分类（...）
- 试试Prompt方法，PET是一种将任务重新编写为完形填空问题，训练时同时利用了有标签和无标签的数据，非常贴合只有少量标注的场景（...）


新增：
- 沿用betterme大佬的规则匹配，使用近义词查找、TF-IDF尽可扩大食品安全相关的主题词，提高规则匹配结果和TextRank的召回率和精确率，大约能有1个点左右的提升。
```python
例如扩增下列词语：

卫生差、卫生不合格、腹泻、肚子不舒服、有烟头、有异物
```
- 替换更严格的正则表达式做规则匹配，大约有2.5个点左右的提升。
```python
正则表达式构造样例：

pattern = '(食品|食物|环境|卫生|店铺|餐食|消防|许可证|健康证).*?(差|问题|隐患|不达标|不过关|不合理|不合格)|问题食品|食品质量|食品安全|有害食品|食品添加剂|强制购物|强制消费|消费维权|欺瞒消费'

df.loc[df["concat"].str.contains(pattern), 'EVENT_TYPE'] = 1
```

    
各base效果：

|方法|分数|
|--- |--- |
|规则匹配（betterme）|94.0610|
|TextRank|91.2730|
|规则匹配（betterme）+ OCSVM|	93.2805|
|扩增食品安全主题词|95.0985|
|扩增食品安全主题词 + OCSVM|	95.3380|
|正则表达式|96.5010|
|正则表达式 + OCSVM|96.8580|

下面是完整代码

In [ ]:
'''
食品以及食品安全的定义：
(1) 食品，指各种供人食用或者饮用的成品和原料以及按照传统既是食品又是中药材的物品，
但是不包括以治疗为目的的物品。 
(2) 食品安全，指食品无毒、无害，符合应当有的营养要求，对人体健康不造成任何急性、亚急性或者慢性危害。
食品安全包括食品卫生、食品质量、食品营养等相关内容和
食品（食物）种植、养殖、加工、包装、储藏、运输、销售、消费等各个环节。
'''

# 0 安装依赖

In [ ]:
!pip install --upgrade pip
!pip install hyperopt

# 1 规则匹配

规则匹配的代码沿用了大佬的代码

在此感谢Betterme大佬开源超简洁的[厦门大数据安全开放创新应用大赛-食品安全专题/baseline_0.94.py](https://github.com/yuanjie-ai/AIZoo/blob/master/__CompetitionBaseline/%E5%8E%A6%E9%97%A8%E5%A4%A7%E6%95%B0%E6%8D%AE%E5%AE%89%E5%85%A8%E5%BC%80%E6%94%BE%E5%88%9B%E6%96%B0%E5%BA%94%E7%94%A8%E5%A4%A7%E8%B5%9B-%E9%A3%9F%E5%93%81%E5%AE%89%E5%85%A8%E4%B8%93%E9%A2%98/baseline_0.94.py)

In [5]:
import os
import pandas as pd
from tqdm import tqdm
# 读取数据
file_dir = r"./work/data/食品安全-算法分析题初赛A榜-综合信息数据.xls"
df = pd.read_excel(file_dir).fillna('')
df.rename(columns={'EVEN_ID': 'EVENT_ID'}, inplace=True)
df["concat"] = [str(row['EVENT_NAME'])+" "+str(row['CONTENT']) for idx,row in df.iterrows()]
df['EVENT_TYPE'] = 0
# 规则匹配
pattern = '食|餐|肉|饭|菜|面包|蛋糕'
df.loc[df["concat"].str.contains(pattern), 'EVENT_TYPE'] = 1

print(df['EVENT_TYPE'].value_counts())
os.makedirs('./work/prediction/Relu',exist_ok=True)
df[["EVENT_ID","EVENT_TYPE","EVENT_NAME"]].to_csv("./work/prediction/Relu/result.txt",index=False,encoding="utf-8",sep="|")

# 得分 94.0610

0    5500
1    1500
Name: EVENT_TYPE, dtype: int64


# 2 基于TextRank的规则匹配

In [6]:
# TextRank
import jieba
import jieba.analyse as analyse

# copy一份dataframe
tr_df = df.copy()
# 停用词表
stopwords_files = [r"./work/stopwords/baidu_stopwords.txt",]
# 读取停用词
for i in stopwords_files:
    analyse.set_stop_words(i)
# 提取TextRank关键词,筛选分数最高的前Top K个关键词
for ids,sentence in enumerate(tr_df["concat"]):
    # TextRank关键词提取，词性筛选
    keywords = analyse.textrank(sentence, topK=3, allowPOS=('n', 'nz', 'v', 'vd', 'vn', 'l', 'a', 'd'))
    tr_df.loc[ids, "TEXTRANK_WORD"] = " ".join(keywords)
# TextRank关键词和pattern进行匹配，筛选出食品安全相关信息
tr_df['EVENT_TYPE'] = 0
tr_df.loc[tr_df["TEXTRANK_WORD"].str.contains(pattern), 'EVENT_TYPE'] = 1
# 得到TextRank结果
df['TEXTRANK_EVENT_TYPE'] = tr_df['EVENT_TYPE']
print(df['TEXTRANK_EVENT_TYPE'].value_counts())
# 保存结果
os.makedirs('./work/prediction/TextRank',exist_ok=True)
df.to_csv("./work/prediction/TextRank/result.txt",index=False,encoding="utf-8",sep="|")

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.752 seconds.
Prefix dict has been built successfully.


0    6205
1     795
Name: TEXTRANK_EVENT_TYPE, dtype: int64


# 3 句向量特征提取

In [7]:
import paddle
from functools import partial
from paddlenlp.data import Stack, Dict, Pad
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import BertModel, BertTokenizer

# 特征提取器
class Feature_extract(object):
    # 初始化
    def __init__(self,tokenizer,model,mode,batch_size=128):
        self.tokenizer = tokenizer
        self.model = model
        self.mode = mode
        self.batch_size = batch_size

    # 以batch制作输入
    def batch_tokens(self,sentences):
        # 迭代数据函数
        def read(sentences):
            for sentence in sentences:
                yield {"text":sentence}
        # 编码数据函数
        def convert_example(examples, tokenizer, max_seq_len=512):
            tokenized_input = tokenizer(examples['text'],is_split_into_words=True,max_seq_len=max_seq_len)
            return tokenized_input
        # 制作batch数据
        data_ds = load_dataset(read, sentences=sentences, lazy=False)
        data_trans_func = partial(convert_example,tokenizer=self.tokenizer)
        data_ds.map(data_trans_func, lazy=False)
        data_batch_sampler = paddle.io.BatchSampler(data_ds, batch_size=self.batch_size, shuffle=False)
        # 定义batchify_fn
        batchify_fn = lambda samples, fn = Dict({
            "input_ids": Pad(axis=0, pad_val=self.tokenizer.pad_token_id), 
            "token_type_ids": Pad(axis=0, pad_val=self.tokenizer.pad_token_type_id),
        }): fn(samples)
        # 制作dataloader
        data_loader = paddle.io.DataLoader(
            dataset=data_ds,
            batch_sampler=data_batch_sampler,
            collate_fn=batchify_fn,
            return_list=True)
        return data_loader

    # 模型推理，获取特征
    @paddle.no_grad()
    def prepare_features(self,examples):
        outputs = []
        self.model.eval()
        data_loader = self.batch_tokens(examples)
        for batch in tqdm(data_loader):
            output = self.model(input_ids=batch[0],token_type_ids=batch[1])
            if self.mode == "cls":
                output = output[1]
            if self.mode == "last_avg":
                output = output[0].mean(axis=1)
            outputs.append(output)
        outputs = paddle.concat(outputs,axis=0)
        return outputs
    
    # 特征提取函数
    def get_emb(self,setences):
        return self.prepare_features(setences)

# 超参数
max_seq_length = 64
mode = "cls"   # cls | last_avg
# 特征提取并制作训练数据
# 使用simbert作为特征提取器的骨干模型
tokenizer = BertTokenizer.from_pretrained('simbert-base-chinese')
sim_model = BertModel.from_pretrained('simbert-base-chinese')
fe = Feature_extract(tokenizer,sim_model,mode=mode,batch_size=250)
# 创建训练集（只含正例），只使用TextRank筛选出的食品安全句子作为正例
train = df.copy()
pos_data = train[train['TEXTRANK_EVENT_TYPE'] == 1]
pos_data.reset_index(drop=True,inplace=True)
train_pos_X = fe.get_emb(pos_data['concat'].str.replace("*",""))
print(train_pos_X.shape)

# 将全量数据embedding化用于预测,规则提取得到的标签可以视作伪标签
train_X =  fe.get_emb(df['concat'].str.replace("*", ""))
train_y = df['EVENT_TYPE']

[2022-09-29 13:52:57,919] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/simbert/vocab.txt and saved to /home/aistudio/.paddlenlp/models/simbert-base-chinese
[2022-09-29 13:52:57,921] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/simbert/vocab.txt
100%|██████████| 64/64 [00:00<00:00, 14911.42it/s]
[2022-09-29 13:52:57,994] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/simbert/simbert-base-chinese-v1.pdparams and saved to /home/aistudio/.paddlenlp/models/simbert-base-chinese
[2022-09-29 13:52:57,996] [    INFO] - Downloading simbert-base-chinese-v1.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/simbert/simbert-base-chinese-v1.pdparams
100%|██████████| 630272/630272 [00:09<00:00, 65408.83it/s]
W0929 13:53:07.785506   167 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0929 13:53:07.7901

[795, 768]


100%|██████████| 28/28 [00:54<00:00,  2.27s/it]


# 4 训练OneClassSVM分类器

In [8]:
from sklearn import svm
from sklearn.metrics import f1_score

# 先测试一下OCSVM的效果，然后超参数搜索得到更好的分数
clf = svm.OneClassSVM(nu=0.01, kernel='rbf',gamma=0.01)
clf.fit(train_pos_X)
y_pred_train = clf.predict(train_X)
y_pred_train = [0 if i==-1 else 1 for i in y_pred_train]
f1s = f1_score(y_true=train_y, y_pred=y_pred_train, average='binary')
df['PRE_EVENT_TYPE'] = y_pred_train
print('f1:',f1s)
print(df['PRE_EVENT_TYPE'].value_counts())

f1: 0.8371268105232041
0    5117
1    1883
Name: PRE_EVENT_TYPE, dtype: int64


# 5 超参数搜索

In [9]:
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial, STATUS_OK
from hyperopt.early_stop import no_progress_loss
from hyperopt import fmin, tpe, hp,  Trials

def hyperopt_train_test(params):
    clf = svm.OneClassSVM(kernel='rbf',nu=params['nu'],gamma=params['gamma'])
    clf.fit(train_pos_X)
    y_pred_train = clf.predict(train_X)
    y_pred_train = [0 if i==-1 else 1 for i in y_pred_train]
    f1s = f1_score(y_true=train_y, y_pred=y_pred_train, average='binary')
    return f1s

space_params = {
    'nu': hp.uniform('nu', 0.,0.1),
    'gamma': hp.uniform('gamma', 0.,0.1),
}

def hyperopt_objective(params):
    f1 = hyperopt_train_test(params)
    return {'loss': -f1, 'status': STATUS_OK}
early_stop_fn = no_progress_loss(100)

trials = Trials()
best = fmin(hyperopt_objective,
            space_params,
            algo = tpe.suggest,
            max_evals = 200,
            trials = trials,
            verbose = True,
            early_stop_fn = early_stop_fn)
print('best:',best)

 61%|██████    | 122/200 [02:15<01:51,  1.43s/trial, best loss: -0.8882255389718077]
best: {'gamma': 0.018130049917692855, 'nu': 0.04651376476124187}


In [15]:
"""
best: {'gamma': 0.018023733194832452, 'nu': 0.02380911877940181, 'f1':0.8885941644562334}
best: {'gamma': 0.018130049917692855, 'nu': 0.04651376476124187, 'f1':0.8882255389718077}
best: {'gamma': 0.017305657389549374, 'nu': 0.04523303959300258, 'f1':0.8859677949392046}
"""

"\nbest: {'gamma': 0.018023733194832452, 'nu': 0.02380911877940181, 'f1':0.8885941644562334}\nbest: {'gamma': 0.018130049917692855, 'nu': 0.04651376476124187, 'f1':0.8882255389718077}\nbest: {'gamma': 0.017305657389549374, 'nu': 0.04523303959300258, 'f1':0.8859677949392046}\n"

In [11]:
# 搜索是随机，可能每次运行都不太一样，做好记录
search_best_params = {'gamma': 0.018023733194832452, 'nu': 0.02380911877940181,'f1':0.8885941644562334}

# 6 最佳模型训练和预测

In [41]:
# 使用最佳参数训练模型并得到结果
clf = svm.OneClassSVM(kernel='rbf', nu=search_best_params['nu'], gamma=search_best_params['gamma'])
clf.fit(train_pos_X)
y_pred_train = clf.predict(train_X)
y_pred_train = [0 if i==-1 else 1 for i in y_pred_train]
df['EVENT_TYPE'] = y_pred_train
print(df['EVENT_TYPE'].value_counts())
os.makedirs('./work/prediction/OCSVM',exist_ok=True)
df[["EVENT_ID","EVENT_TYPE","EVENT_NAME"]].to_csv("./work/prediction/OCSVM/result.txt",index=False,encoding="utf-8",sep="|")

# 93.2805

0    5494
1    1506
Name: EVENT_TYPE, dtype: int64


In [38]:
# # 初始化记录器，visualdl不知道什么原因一直在加载，只能手写一个UMAP 3D观察向量空间中样本的变化
# from visualdl import LogWriter
# with LogWriter(logdir='./log/token_hidi') as writer:
#     writer.add_embeddings(tag='test', mat=[i for i in train_X.numpy()], metadata=df['EVENT_TYPE'].astype(str).tolist())
# import numpy as np
# np.save('paddle_trian_x.npy',train_X.numpy())
# np.save('paddle_trian_y.npy',df['EVENT_TYPE'].astype(str).tolist())

paddle自带的visualdl不知道什么原因一直加载不出来，只能手写一个UMAP3D观察向量空间中样本的分布

从图片中看，句向量在空间中的聚类效果很明显

其中蓝色是预测的非食安信息，红色的是食安信息

- 蓝色小团簇，非食安信息，以 转医院-无名氏-疾病类 为代表
- 蓝色大团簇，非食安信息
- 红色团簇，食安信息

![plot](https://ai-studio-static-online.cdn.bcebos.com/a03ae46b7e944888b7fab061282fdabcd308b42cbb3c4188a288e345d9b108fc)

![蓝色小簇](https://ai-studio-static-online.cdn.bcebos.com/722a5c6f0a8f474d86e241edaf5ec932c01ad416084e42c09ce9fc9afe89bdac)

![蓝色大簇](https://ai-studio-static-online.cdn.bcebos.com/9e7447825fa146279db6c41892ed600e081b6dfc47004f5c8b1b72746dfa7101)

![红色簇](https://ai-studio-static-online.cdn.bcebos.com/a0873aaaca80481fbf0b97d94573a997b09ac8d75ab54e2ea9c21bcb9938b651)

总结：

思路上摒弃了常规的人工少量标注 + 文本分类，转而使用带边界的异常检测分类器来做文本分类，希望借助边界的力量，排除非食安信息。


作者:Armor

我在AI Studio上获得钻石等级，点亮8个徽章，来互关呀~ https://aistudio.baidu.com/aistudio/personalcenter/thirdview/392748